In [2]:
import pandas as pd
import requests
import json

In [5]:
data = pd.read_csv("Resources/State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
data.head()
data["RegionID", "RegionType", "RegionName"]

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31
0,9,0,California,State,CA,202744.0,203606.0,204690.0,206971.0,209417.0,...,677568.0,696320.0,710135.0,720036.0,725067.0,730580.0,737410.0,747660.0,759840.0,774899.0
1,54,1,Texas,State,TX,114109.0,114234.0,114273.0,114480.0,114529.0,...,251448.0,257104.0,262775.0,267628.0,271901.0,275537.0,279800.0,284463.0,289905.0,295319.0
2,43,2,New York,State,NY,148975.0,149737.0,150374.0,151718.0,152933.0,...,352114.0,357721.0,362532.0,366549.0,369590.0,372459.0,375299.0,378578.0,382341.0,387069.0
3,14,3,Florida,State,FL,116274.0,116617.0,116980.0,117685.0,118355.0,...,296250.0,304232.0,312319.0,320255.0,327674.0,335023.0,341864.0,349456.0,357423.0,367175.0
4,21,4,Illinois,State,IL,147631.0,147955.0,148395.0,149261.0,150153.0,...,232170.0,235639.0,239047.0,241589.0,243829.0,246095.0,248604.0,250960.0,253330.0,256010.0
